# <span style="color: #FF0000">Exemple de machine de turing</span>

In [32]:
from ipywidgets import Button, Textarea, Layout,Label,HBox,HTML,Dropdown
from IPython.display import display,clear_output
import time

### Ajouter 1 à un binaire

In [2]:
Etat = {}
e1= {}
e1["0"] = ("e1","0","0","droite","e2")
e1["1"] = ("e1","1","1","droite","e2")
e1[" "] = ("e1"," "," ","droite","e1")
e2 = {}
e2["0"] = ("e2","0","0","droite","e2")
e2["1"] = ("e2","1","1","droite","e2")
e2[" "] = ("e2"," "," ","gauche","e3")
e3 = {}
e3["0"] = ("e3","0","1","gauche","fin")
e3["1"] = ("e3","1","0","gauche","e3")
e3[" "] = ("e3"," ","1","droite","fin")

Etat["fin"]=None
Etat["e1"]=e1
Etat["e2"]=e2
Etat["e3"]=e3

liste = []
for k, v in Etat.items():
    if k!="fin":
        for k2, v2 in v.items():
            b = "|".join(v2)
            b=b.replace(" ","VIDE")
            liste.append("|"+b+"|")
a = "|Etat|Lit|Ecrit|Déplace|Suivant|\n|---|---|---|---|---|\n"+"\n".join(liste)

{{print(a)}}

In [1]:
def on_value_change(change=None,i=None):
    global Ruban,dd,Rubani
    Rubani[i]=dd[i].value
    
def on_button_click(b):
    global go,Ruban,Rubani,titre,eiw,dd,step,e,c
    clear_output()
    #display(titre)
    display(eiw)
    #display(HBox([go,step]))
    fin = True
    for el in Rubani:
        if el!=" ":
            fin = False
            break
    if fin:
        return
    for el in dd:
        el.disabled=True
    go.disabled=True
    step.disabled=True
    Ruban=Rubani[:]
    affichage()
    turing(e,c)
    go.disabled=False
    step.disabled=False
    for el in dd:
        el.disabled=False
    dd[0].disabled=True
    dd[-1].disabled=True
    e = "e1"
    c = 0
    
def unpas(b):
    global go,Ruban,Rubani,titre,eiw,dd,step,e,c
    clear_output()
    #display(titre)
    display(eiw)
    #display(HBox([go,step]))
    fin = True
    for el in Rubani:
        if el!=" ":
            fin = False
            break
    if fin:
        return
    for el in dd:
        el.disabled=True
    go.disabled=True
    step.disabled=True
    Ruban=Rubani[:]
    affichage()
    e,c = turing(e,c,auto=False)
    go.disabled=False
    step.disabled=False
    if e =="fin":
        for el in dd:
            el.disabled=False
        dd[0].disabled=True
        dd[-1].disabled=True
        e = "e1"
        c = 0
    
    
def etatinit():
    global Ruban,dd,go,Rubani,titre,eiw,step,e,c
    Rubani=[" " for i in range(10)]
    Rubani[1]="1"
    Rubani[2]="0"
    Rubani[3]="1"
    e="e1"
    c=0
    dd = []
    for i in range(10):
        dd.append(Dropdown(
    options=[' ', '0', '1'],
    value=Rubani[i],
    description='',
    disabled=False,
    layout=Layout(width='50px',height='50px'),
))
    dd[0].disabled=True
    dd[-1].disabled=True
    
    titre=HTML(value = f"<center><b><font color='black'>Etat Initial</b></center>", layout = Layout(width='200px',height='50px'))
    
    for i in range(10):
        dd[i].observe(lambda change,x=i:on_value_change(change,x),names="value")
    go = Button(description="GO",disabled=False)
    step = Button(description = "1 pas",disabled=False)
    eiw=HBox([titre]+dd+[go,step])
    #display(titre)
    display(eiw)
    #display(HBox([go,step]))
    go.on_click(on_button_click)
    step.on_click(unpas)
            
def affichage():
    global Etat,Ruban,wcases,wetat,etatlab,entete,wentete,wetat,wcases,wruban
    titre2=HTML(value = f"<center><b><font color='black'>Machine de Turing</b></center>", layout = Layout(width='200px',height='50px'))
    #display(titre2)
    
    entete = []
    enteteval =[" ","lit","ecrit","deplace","suivant"]
    for i in range(5):
        entete.append(HTML(value = f"<center><b><font color='black'>"+enteteval[i]+"</b></center>", layout = Layout(width='50px',height='50px')))
    wentete = HBox(entete)
    display(wentete)                  
    etatval = ["e1","1","1","droite","e1"]
    etatlab = []
    for i in range(5):
        etatlab.append(HTML(value = f"<center><b><font color='black'>"+etatval[i]+"</b></center>", layout = Layout(width='50px',height='50px')))
    wetat = HBox(etatlab)
    display(wetat)
    wcases = []
    for i in range(len(Ruban)):
        wcases.append(HTML(value = f"<center><b><font color='black'>"+Ruban[i]+"</b></center>", layout = Layout(width='50px',height='50px',border='3px solid black',)))
    wruban = HBox([titre2]+wcases)
    display(wruban)

def turing(etat = "e1", case = 0, auto=True):
    global Etat,Ruban,wcases,wetat,etatlab
    #etat = Etat[e][Ruban[case]]
    etatval = Etat[etat][Ruban[case]]
    for i in range(5):
        etatlab[i].value = f"<center><b><font color='black'>"+etatval[i]+"</b></center>"
    for i in range(len(wcases)):
        wcases[i].layout.border = '3px solid black'

    wcases[case].layout.border = '3px solid red'
    
    for i in range(1,5):
        etatlab[i-1].layout.border = '0px'
        etatlab[i].layout.border = '3px solid red'
        time.sleep(0.25)
        if i==2:
            Ruban[case]=etatval[2]
            wcases[case].value = f"<center><b><font color='black'>"+Ruban[case]+"</b></center>"
        elif i == 3:
            wcases[case].layout.border = '3px solid black'
            if etatval[3]=="droite":
                case+=1
            else:
                case-=1
            wcases[case].layout.border = '3px solid red'
        elif i==4:
            etat = etatval[4]
        time.sleep(0.5)
    
    if etat == "fin":
        etatlab[-1].layout.border = '0px'
        wcases[case].layout.border = '3px solid black'
        return (etat,case)
    else:
        etatlab[-1].layout.border = '0px'
        if auto:
            turing(etat,case)
        return (etat,case)

etatinit()

NameError: name 'Dropdown' is not defined